In [1]:
# 기본 Library 설정
import numpy as np
import statistics
import random

import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import bytespdate2num
import matplotlib.ticker as ticker
from matplotlib import font_manager, rc
from matplotlib import style
import seaborn as sns

import calendar
import dateutil
from dateutil.parser import parse
import datetime

# Clustering
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram


# 한글 사용하기
import matplotlib.font_manager as fm
from matplotlib import rc
font_name = fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

from tqdm import tqdm
import time

# from zeep import Client

from collections import namedtuple
import sqlite3

# 한 번에 matplotlib 그림 띄우기
%matplotlib inline    
%config InlineBackend.figure_format = 'retina'  #%matplotlib 뒤에 써주면 그래프를 더 높은 해상도로 보여줌

# 설정 관련
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# 0. 수요예측하기

In [2]:
# Data 전부 불러주기
pck_total_data = pd.read_csv('data/(Main)고객주문통합리스트_세부품목분류완료.csv', encoding='euc-kr')
pck_royal_cst = pd.read_csv('data/(Main)2019이후_3회_이상_주문한_사람들.csv', encoding='euc-kr')

c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (28,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (32,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pck_royal_cst['주문자ID'].nunique(), pck_total_data['주문자ID'].nunique()

(1462, 4969)

In [4]:
### 새로운 column들 추가 ###
# Timeseries로 바꿔주고!
pck_total_data['주문일시']= pd.to_datetime(pck_total_data['주문일시'])
pck_total_data['결제일시(입금확인일)']= pd.to_datetime(pck_total_data['결제일시(입금확인일)'])
pck_total_data['발주일']= pd.to_datetime(pck_total_data.발주일)
pck_total_data['배송대기일']= pd.to_datetime(pck_total_data.배송대기일)
pck_total_data['배송시작일']= pd.to_datetime(pck_total_data.배송시작일)
pck_total_data['배송완료일']= pd.to_datetime(pck_total_data.배송완료일)
pck_total_data['주문자 가입일']= pd.to_datetime(pck_total_data['주문자 가입일'])
pck_total_data['가장최근주문']= pd.to_datetime(pck_total_data['가장최근주문'])
del pck_royal_cst['세부분류사이즈']

pck_royal_cst['주문일시']= pd.to_datetime(pck_royal_cst['주문일시'])
pck_royal_cst['결제일시(입금확인일)']= pd.to_datetime(pck_royal_cst['결제일시(입금확인일)'])
pck_royal_cst['발주일']= pd.to_datetime(pck_royal_cst.발주일)
pck_royal_cst['배송대기일']= pd.to_datetime(pck_royal_cst.배송대기일)
pck_royal_cst['배송시작일']= pd.to_datetime(pck_royal_cst.배송시작일)
pck_royal_cst['배송완료일']= pd.to_datetime(pck_royal_cst.배송완료일)
pck_royal_cst['주문자 가입일']= pd.to_datetime(pck_royal_cst['주문자 가입일'])
pck_royal_cst['가장최근주문']= pd.to_datetime(pck_royal_cst['가장최근주문'])

# pck_total_data와 pck_royal_cst에 세부분류사이즈, 총주문량, 주문연도.월 columns 만들어주기
pck_total_data.loc[(pck_total_data['세부분류'].isna()==False)&(pck_total_data['사이즈'].isna()==False), '세부분류사이즈'] = pck_total_data['세부분류']+"_"+pck_total_data['사이즈']
pck_total_data.loc[(pck_total_data['세부분류'].isna()==True)&(pck_total_data['사이즈'].isna()==False), '세부분류사이즈'] = pck_total_data['사이즈']
pck_total_data.loc[(pck_total_data['세부분류'].isna()==False)&(pck_total_data['사이즈'].isna()==True), '세부분류사이즈'] = pck_total_data['세부분류']
pck_total_data.loc[(pck_total_data['세부분류'].isna()==True)&(pck_total_data['사이즈'].isna()==True), '세부분류사이즈'] = None
pck_total_data['세부분류사이즈'] = pck_total_data['세부분류사이즈'].replace("일회용 숟가락_175mm","일회용 숟가락")    # 일회용 숟가락은 사이즈에 상관없이 통일시켜주기
pck_total_data['세부분류사이즈'] = pck_total_data['세부분류사이즈'].replace("일회용 숟가락_170mm","일회용 숟가락")    # 일회용 숟가락은 사이즈에 상관없이 통일시켜주기
pck_total_data['세부분류사이즈'] = pck_total_data['세부분류사이즈'].str.strip()    # 띄어쓰기 때문에 다른 품목으로 인식하는 경우 방지하기 위해서 공백 없애기
pck_total_data['총주문량'] = pck_royal_cst['총주문량']
pck_total_data['주문연도.월'] = pck_royal_cst['주문연도.월']


pck_royal_cst.loc[(pck_royal_cst['세부분류'].isna()==False)&(pck_royal_cst['사이즈'].isna()==False), '세부분류사이즈'] = pck_royal_cst['세부분류']+"_"+pck_royal_cst['사이즈']
pck_royal_cst.loc[(pck_royal_cst['세부분류'].isna()==True)&(pck_royal_cst['사이즈'].isna()==False), '세부분류사이즈'] = pck_royal_cst['사이즈']
pck_royal_cst.loc[(pck_royal_cst['세부분류'].isna()==False)&(pck_royal_cst['사이즈'].isna()==True), '세부분류사이즈'] = pck_royal_cst['세부분류']
pck_royal_cst.loc[(pck_royal_cst['세부분류'].isna()==True)&(pck_royal_cst['사이즈'].isna()==True), '세부분류사이즈'] = None
pck_royal_cst['세부분류사이즈'] = pck_royal_cst['세부분류사이즈'].replace("일회용 숟가락_175mm","일회용 숟가락")    # 일회용 숟가락은 사이즈에 상관없이 통일시켜주기
pck_royal_cst['세부분류사이즈'] = pck_royal_cst['세부분류사이즈'].replace("일회용 숟가락_170mm","일회용 숟가락")    # 일회용 숟가락은 사이즈에 상관없이 통일시켜주기
pck_royal_cst['세부분류사이즈'] = pck_royal_cst['세부분류사이즈'].str.strip()    # 띄어쓰기 때문에 다른 품목으로 인식하는 경우 방지하기 위해서 공백 없애기

# 쓰는 column만 [pck]로 추려주기
pck = pck_royal_cst[['주문번호', '주문자ID', '총 결제금액', '주문연도.월','세부분류사이즈', '총주문량', 
                    '개수', '수량','세부분류', '주문일시','19년이후 누적주문횟수',
                    '상품번호']]

# 총 결제금액의 중첩을 피하기 위해서 중첩된 주문일시는 빼주기
pck_dropdup = pck.drop_duplicates(['주문일시'])
pck_royal_cst_dropdup = pck_royal_cst.drop_duplicates(['주문일시'])
pck_total_data_dropdup = pck_total_data.drop_duplicates(['주문일시'])

In [5]:
pck_total_data

,주문번호,주문자ID,주문경로(PC/모바일),결제수단,총 결제금액,주문상품명,상품번호,주문횟수,상품명(패커스(PACKUS)),수량,분류,세부분류,사이즈,색상,개수,공급사,자체상품코드,주문일시,결제일시(입금확인일),발주일,배송대기일,배송시작일,배송완료일,세금구분,공급원가,상품 매입가,판매가,상품구매금액,상품별 추가할인 상세,옵션+판매가,쿠폰 할인금액,주문자 가입일,특별관리회원,회원추가항목_가입경로,회원추가항목_업종,주문 시 회원등급,주문자 주소,수령인 주소,위도,경도,주문-배송완료,발주일-배송시작일,배송시작-배송완료,회원가입연도,회원가입월,회원가입일,가장최근주문,19년이후 누적주문횟수,가입기간(일수),세부분류사이즈,총주문량,주문연도.월
0,20170101-0000015,cgh1385,모바일,무통장입금,70500,"105파이 다용도컵 세트 5호 대 370ml 백색 PS 1,000개 1set(2박스)",45,398.0,"105파이 다용도컵 세트 5호 대 370ml 백색 PS 1,000개 1set(2박스)",1,30.0,다용도컵,370ml,백색,2000.0,패커스,P00000BS,2017-01-01 15:38:24,2017-01-02 16:38:58,2017-01-02 16:38:58,2017-01-17 08:25:36.860,2017-01-17 08:26:04,2017-01-17 08:26:26,과세상품,51700.0,66500,66500,66500.0,NaN,66500.0,0.0,2017-01-01,NaN,NaN,NaN,일반,전라북도 전주시 덕진구 동부대로 879 (호성동1가),전라북도 전주시 덕진구 동부대로 879 (호성동1가),35.869479,127.157255,15 days 16:48:02.000000000,14 days 15:47:06.000000000,0 days 00:00:22.000000000,2017,01,01,2017-01-01 15:38:24,NaN,1004,다용도컵_370ml,6000.0,2017_3
1,20170115-0000013,pjkl77,PC,"쿠폰,카드",16000,소량 105파이 다용도컵 세트 5호 대 370ml 투명 PS 100개,67,6.0,소량 105파이 다용도컵 세트 5호 대 370ml 투명 PS 100개,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-15 01:24:43,2017-01-15 01:24:43,2017-01-15 01:24:43,2017-01-19 08:56:18.848,2017-01-19 08:56:55,2017-01-19 19:51:02,과세상품,6200.0,17000,17000,17000.0,NaN,17000.0,5000.0,2017-01-14,NaN,NaN,NaN,일반,전라북도 전주시 완산구 효자동3가 1676-13,전라북도 전주시 완산구 효자동3가 1676-13,35.819967,127.095559,4 days 18:26:19.000000000,4 days 07:32:12.000000000,0 days 10:54:07.000000000,2017,01,01,2017-02-26 22:55:59,NaN,1004,None,4000.0,2017_3
2,20170123-0000021,pjkl77,PC,"쿠폰,카드",18000,소량 105파이 다용도컵 세트 5호 대 370ml 투명 PS 100개,67,6.0,소량 105파이 다용도컵 세트 5호 대 370ml 투명 PS 100개,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-23 00:35:04,2017-01-23 00:35:04,2017-01-23 00:35:04,2017-01-23 13:38:25.188,2017-01-23 15:26:06,2017-01-24 18:00:13,과세상품,6200.0,17000,17000,17000.0,NaN,17000.0,3000.0,2017-01-14,NaN,NaN,NaN,사업자,전라북도 전주시 완산구 효자동3가 1676-13,전라북도 전주시 완산구 효자동3가 1676-13,35.819967,127.095559,1 days 17:25:09.000000000,0 days 14:51:02.000000000,1 days 02:34:07.000000000,2017,01,01,2017-02-26 22:55:59,NaN,1004,None,4000.0,2017_4
3,20170226-0000015,pjkl77,PC,페이나우(카드),21000,소량 105파이 다용도컵 세트 5호 대 370ml 투명 PS 100개,67,6.0,소량 105파이 다용도컵 세트 5호 대 370ml 투명 PS 100개,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-26 22:55:59,2017-02-26 22:55:59,2017-02-26 22:55:59,2017-02-27 11:06:40.677,2017-02-27 15:26:17,2017-03-01 09:45:30,과세상품,6200.0,17000,17000,17000.0,NaN,17000.0,0.0,2017-01-14,NaN,NaN,NaN,사업자,전라북도 전주시 완산구 효자동3가 1676-13,전라북도 전주시 완산구 효자동3가 1676-13,35.819967,127.095559,2 days 10:49:31.000000000,0 days 16:30:18.000000000,1 days 18:19:13.000000000,2017,01,01,2017-02-26 22:55:59,NaN,1004,None,6000.0,2017_5
4,20170118-0000019,kim0750,PC,"쿠폰,무통장입금",72700,"105파이 다용도컵 세트 5호 특대 400ml 백색 PS 1,000개 1set(2박스)",46,159.0,"105파이 다용도컵 세트 5호 특대 400ml 백색 PS 1,000개 1set(2박스)",1,30.0,다용도컵,400ml,백색,2000.0,패커스,P00000BT,2017-01-18 16:41:57,2017-01-18 18:38:50,2017-01-18 18:38:50,2017-01-19 19:54:17.175,2017-01-19 19:55:09,2017-01-20 16:14:06,과세상품,58300.0,73700,73700,73700.0,NaN,73700.0,5000.0,2017-01-18,NaN,NaN,NaN,사업자,경상북도 구미시 송동로7길 31-3 (도량동),경상북도 구미시 송동로7길 31-3 (도량동),36.145442,128.320529,1 days 23:32:09.000000000,1 days 01:16:19.000000000,0 days 20:18:57.000000000,2017,01,01,2017-01-18 16:41:57,NaN,1004,다용도컵_400ml,6000.0,2017_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49441,1908291803208274,bontofood,NaN,무통장 입금,109970,"105파이 다용도컵 세트 5호 중 300ml 블랙 PS-1,000개 1set(2박스...",447,396.0,PP 5칸 도시락용기 04 200개 1박스(작은 사이즈),2,4.0,도시락용기,5칸,NaN,200.0,패커스,P00000RE,2019-08-29 18:03:21,2019-09-02 09:00:56,NaT,NaT,NaT,NaT,NaN,NaN,36300,48500,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,새내기,NaN,인천광역시 남동구 함박뫼로347번길 53 (남촌동),37.415597,126.703537,NaN,NaN,NaN,NaT,NaT,NaT,2019-08-29 18:03:21,61.0,1004,도시락용기_5칸,NaN,NaN
49442,1908291323213045,NaN,NaN,무통장 입금,172700,125파이 원형용기 소 25

In [6]:
pck_royal_cst

,주문번호,주문자ID,주문경로(PC/모바일),결제수단,총 결제금액,주문상품명,상품번호,주문횟수,상품명(패커스(PACKUS)),수량,분류,세부분류,사이즈,색상,개수,공급사,자체상품코드,주문일시,결제일시(입금확인일),발주일,배송대기일,배송시작일,배송완료일,세금구분,공급원가,상품 매입가,판매가,상품구매금액,상품별 추가할인 상세,옵션+판매가,쿠폰 할인금액,주문자 가입일,특별관리회원,회원추가항목_가입경로,회원추가항목_업종,주문 시 회원등급,주문자 주소,수령인 주소,위도,경도,주문-배송완료,발주일-배송시작일,배송시작-배송완료,회원가입연도,회원가입월,회원가입일,가장최근주문,19년이후 누적주문횟수,가입기간(일수),누적주문,주문연도.월,총주문량,세부분류사이즈
0,20170323-0000017,xigol80,PC,"쿠폰,카드",169500,"70파이 다용도컵 세트 1호 소 70ml 백색 PS 3,000개 1set(2박스)",38,477.0,"70파이 다용도컵 세트 1호 소 70ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,70ml,백색,6000.0,패커스,P00000BL,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 10:32:06.096,2017-03-23 18:43:40,2017-04-01 19:49:03,과세상품,42900.0,40500,53100,53100.0,NaN,53100.0,2000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,9 days 16:39:45.000000000,0 days 15:34:22.000000000,9 days 01:05:23.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_3,6000.0,다용도컵_70ml
1,20170323-0000017,xigol80,PC,"쿠폰,카드",169500,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42,471.0,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",2,30.0,다용도컵,200ml,백색,2000.0,패커스,P00000BP,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 10:33:14.195,2017-03-23 18:43:40,2017-04-01 19:49:03,과세상품,44000.0,43000,59200,118400.0,NaN,59200.0,2000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,9 days 16:39:45.000000000,0 days 15:34:22.000000000,9 days 01:05:23.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_3,4000.0,다용도컵_200ml
2,20170421-0000015,xigol80,PC,"쿠폰,카드",113400,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42,471.0,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",2,30.0,다용도컵,200ml,백색,2000.0,패커스,P00000BP,2017-04-21 03:38:14,2017-04-21 03:38:14,2017-04-21 03:38:14,2017-04-21 17:05:22.809,2017-04-21 17:10:58,2017-04-25 08:23:56,과세상품,44000.0,43000,59200,118400.0,NaN,59200.0,5000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,4 days 04:45:42.000000000,0 days 13:32:44.000000000,3 days 15:12:58.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_4,4000.0,다용도컵_200ml
3,20170519-0000012,xigol80,PC,"쿠폰,카드",208590,"70파이 다용도컵 세트 1호 특소 50ml 백색 PS 3,000개 1set(2박스)",47,274.0,"70파이 다용도컵 세트 1호 특소 50ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,50ml,백색,6000.0,패커스,P00000BU,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 16:01:20.590,2017-05-19 19:50:23,2017-05-27 05:35:19,과세상품,41250.0,40500,53100,53100.0,다용도 10%할인,53100.0,3000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,7 days 23:35:52.000000000,0 days 13:50:56.000000000,7 days 09:44:56.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_5,6000.0,다용도컵_50ml
4,20170519-0000012,xigol80,PC,"쿠폰,카드",208590,"70파이 다용도컵 세트 2호 대 80ml 백색 PS 3,000개 1set(2박스)",39,94.0,"70파이 다용도컵 세트 2호 대 80ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,80ml,백색,6000.0,패커스,P00000BM,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 16:01:20.590,2017-05-19 19:50:23,2017-05-27 05:35:19,과세상품,49500.0,45000,63600,63600.0,다용도 10%할인,63600.0,3000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,7 days 23:35:52.000000000,0 days 13:50:56.000000000,7 days 09:44:56.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_5,6000.0,다용도컵_80ml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33199,1908301422425566,q9691445,NaN,무통장 입금,54850,195파이 탕용기 소 300개 1set(2박스),1077,1366.0,195파이 탕용기 소 300개 1set(2박스),1,1.0,탕용기,195_小,NaN,600.0,패커스,P0000BPK,2019-08-30 14:22:42,2019-09-02 09:03:36,NaT,NaT,NaT,NaT,NaN,NaN,44550,57850,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,새내기,NaN,경상북도 포항시 북구 성실로 21-1,36.065835,129.376172,NaN,NaN,NaN,NaT,NaT,NaT,2019-08-30 14:22:42,20.0,1004,11.0,2019_8,600.0,탕용기_195_小
33200,1908301139376461,pp039,NaN,무통장 입금,155

In [7]:
pck

,주문번호,주문자ID,총 결제금액,주문연도.월,세부분류사이즈,총주문량,개수,수량,세부분류,주문일시,19년이후 누적주문횟수,상품번호
0,20170323-0000017,xigol80,169500,2017_3,다용도컵_70ml,6000.0,6000.0,1,다용도컵,2017-03-23 03:09:18,51.0,38
1,20170323-0000017,xigol80,169500,2017_3,다용도컵_200ml,4000.0,2000.0,2,다용도컵,2017-03-23 03:09:18,51.0,42
2,20170421-0000015,xigol80,113400,2017_4,다용도컵_200ml,4000.0,2000.0,2,다용도컵,2017-04-21 03:38:14,51.0,42
3,20170519-0000012,xigol80,208590,2017_5,다용도컵_50ml,6000.0,6000.0,1,다용도컵,2017-05-19 05:59:27,51.0,47
4,20170519-0000012,xigol80,208590,2017_5,다용도컵_80ml,6000.0,6000.0,1,다용도컵,2017-05-19 05:59:27,51.0,39
...,...,...,...,...,...,...,...,...,...,...,...,...
33199,1908301422425566,q9691445,54850,2019_8,탕용기_195_小,600.0,600.0,1,탕용기,2019-08-30 14:22:42,20.0,1077
33200,1908301139376461,pp039,155220,2019_8,죽용기_920ml,2000.0,1000.0,2,죽용기,2019-08-30 11:39:37,89.0,90
33201,1908301139376461,pp039,155220,2019_8,다용도컵_70ml,12000.0,6000.0,2,다용도컵,2019-08-30 11:39:37,89.0,38
33202,1908300953013712,pp024,162070,2019_8,탕용기_195_大,1200.0,600.0,2,탕용기,2019-08-30 09:53:02,30.0,608


## mother 데이터프레임 만들기
고객별 품목당 하루 수요량

In [8]:
# 평균구매주기 계산하기
# 최종으로 모든 id를 합칠 mother_df 만들어주기
mother_columns = list(pck['세부분류사이즈'].unique())
mother_columns.insert(0, '주문자ID')
mother_df = pd.DataFrame(columns=mother_columns)

for customer_id in tqdm(list(pck['주문자ID'].unique())[:5]):
    # 주문자ID에 따른 temp 데이터프레임 만들어주기
    temp = pck[pck['주문자ID']== customer_id]
    
    # pivot_table 만들어주기
    temp_pivot = temp.pivot_table(index='주문연도.월', columns='세부분류사이즈',
                               values='총주문량', aggfunc=np.sum)
    temp_pivot = temp_pivot.T.fillna(0).reset_index()
    temp_pivot['sum'] = temp_pivot.sum(axis=1)
    
    # 세부품목과 총누적구매량 pivot_table 구하기
    new_pivot = temp_pivot[['세부분류사이즈','sum']]
    
    ## 고객이 최근 1년간 구매한 상품목록
    # product_list 만들어주기
    # 각 고객이 1년동안 주문하지 않았다는 것은 더이상 그 상품을 안 쓰기에 재고를 보유하지 않는다는 것으로 간주
    semi_product_list = [product for product in list(temp['세부분류사이즈'].unique()) if type(product) is str]  
    product_list = []
    for i in semi_product_list:
        df_1 = temp[temp['세부분류사이즈']==i].drop_duplicates(['주문일시']).sort_values(by='주문일시')
        if df_1['주문일시'].iloc[-1] >= datetime.date(2018, 9,1):
            product_list.append(i)

            
   # 각 품목별로 여태까지 주문했던 구매주기 모으기
    product_average_order_period_list = []
    product_total_using_period_list = []
    for product in product_list:
        collect_product_order_period_list = []
        df_specific_product = temp[temp['세부분류사이즈']==product]    # 해당 품목만 있는 데이터프레임 만들어주기
        df_specific_product = df_specific_product.drop_duplicates(['주문일시'])    # 주문일시 겹치는거 없애주기
        df_specific_product = df_specific_product.sort_values(by='주문일시')    # 시간순으로 데이터프레임 정렬하기
        
        # 먼저 상품구매경험이 2회 이상인 사람들의 구매주기 모아주기
        if (df_specific_product.shape[0] > 1):
            for i in range(df_specific_product.shape[0]-1):
                product_order_period = df_specific_product['주문일시'].iloc[i+1] - df_specific_product['주문일시'].iloc[i]
                if product_order_period.days == 0:    # 하루 내 여러 차례 주문한 것들은 한 번 주문한 것으로 간주
                    present_dt = datetime.datetime(year=2019, month=9, day=17)
                    product_order_period = present_dt - df_specific_product['주문일시'].iloc[i]
                    collect_product_order_period_list.append([product, product_order_period]) # 구매주기를 품목과 같이 append
                else:
                    collect_product_order_period_list.append([product, product_order_period]) # 구매주기를 품목과 같이 append
        # 상품구매경험이 1회밖에 없는 사람들의 구매주기는 2019년 9월 17일에서 빼서 계산
        else:  
            for i in range(df_specific_product.shape[0]):
                present_dt = datetime.datetime(year=2019, month=9, day=17)
                product_order_period = present_dt - df_specific_product['주문일시'].iloc[i]
                collect_product_order_period_list.append([product, product_order_period]) # 구매주기를 품목과 같이 append
                
        # 구매주기의 평균 구하기
        total_timedelta = datetime.timedelta(days=0, seconds=0)
        for i in collect_product_order_period_list:
            total_timedelta = total_timedelta + i[1]
        average_order_period = round(total_timedelta.days/len(collect_product_order_period_list),2)
        product_average_order_period_list.append([product, average_order_period])    # 품목별 평균구매주기 구했음
                 
        # 상품 누적사용일 구하기
        if (df_specific_product.shape[0] > 1):
            product_1st_end_period = (df_specific_product['주문일시'].iloc[-1] - df_specific_product['주문일시'].iloc[0]).days
            product_total_using_period_list.append([product, product_1st_end_period + average_order_period])
        else:
            product_1st_end_period = (datetime.datetime(year=2019, month=9, day=17) - df_specific_product['주문일시'].iloc[0]).days
            product_total_using_period_list.append([product, product_1st_end_period + average_order_period])
        
    # 전체 사용주기에서 제품이랑 누적사용일 zip해줘서 dictionary로 만들고
    product = []
    days = []
    for i in product_total_using_period_list:
        product.append(i[0])
        days.append(i[1])
    day_dict = dict(zip(product, days))

    # 마지막으로 final pivot 만들어주기
    new_pivot['전체사용일'] = new_pivot['세부분류사이즈'].map(day_dict)
    new_pivot['하루당 사용량'] = round((new_pivot['sum'] / new_pivot['전체사용일']),2)
    final_pivot = new_pivot[['세부분류사이즈','하루당 사용량']]

    # mother_df에 concat할 baby_data 만들어주기
    baby_data = dict(zip(final_pivot['세부분류사이즈'].values, final_pivot['하루당 사용량'].values))
    baby_data['주문자ID'] = customer_id
    baby_df = pd.DataFrame(baby_data, columns=list(baby_data.keys()), index=[0])
    
    # mother_df에 concat해주기
    mother_df = pd.concat([mother_df, baby_df])


  0%|                                                                                         | 0/1462 [00:00<?, ?it/s]c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:91: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas 

In [9]:
temp

,주문번호,주문자ID,총 결제금액,주문연도.월,세부분류사이즈,총주문량,개수,수량,세부분류,주문일시,19년이후 누적주문횟수,상품번호
33041,20190515-0000255,pss00700,52400,2019_5,중화면용기_小,300.0,300.0,1,중화면용기,2019-05-15 10:01:28,3.0,1373
33042,20190516-0000775,pss00700,74990,2019_5,중화면용기_大,300.0,300.0,1,중화면용기,2019-05-16 13:29:53,3.0,1375
33043,20190518-0000029,pss00700,60900,2019_5,중화면용기_中,300.0,300.0,1,중화면용기,2019-05-18 07:46:16,3.0,1374


In [10]:
# column 순서 이쁘게 정리하기
mother_df = mother_df[[ '주문자ID','감자탕용기_2200ml', '감자탕용기_2700ml', '감자탕용기_3200ml', '고강도미니탕용기_미니', '고무장갑', '군만두용기_1칸', '냄비_2750ml', '냄비뚜껑_大', '냉면용기_中', '냉면용기_大', '다용도컵_150ml', '다용도컵_165ml', '다용도컵_200ml', '다용도컵_250ml', '다용도컵_270ml', '다용도컵_300ml', '다용도컵_370ml', '다용도컵_400ml', '다용도컵_50ml', '다용도컵_600ml', '다용도컵_70ml', '다용도컵_80ml', '단무지', '도시락용기(돈까스)_5칸', '도시락용기(돈까스)_中', '도시락용기_4칸 A', '도시락용기_4칸 B', '도시락용기_5칸', '도시락용기_6칸 02', '도시락용기_6칸 L1', '도시락용기_7칸', '도시락용기_8칸', '랩칼', '미니탕용기_미니', '반찬용기_104_1칸', '반찬용기_105_1칸', '반찬용기_201_1칸', '반찬용기_202-2_2칸', '반찬용기_202_1칸', '반찬용기_203_1칸', '반찬용기_204_1칸', '반찬용기_211_1칸', '반찬용기_D-363', '반찬용기_D-364_4칸', '반찬용기_D-365_5칸', '비닐봉투(맛난거)_中', '비닐봉투(맛난거)_大', '비닐봉투(맛난거)_小', '비닐봉투(맛난거)_특大', '비닐봉투(무지)_中', '비닐봉투(무지)_大', '비닐봉투(무지)_小', '비닐봉투(무지)_특大', '비닐봉투(배달중)_中',
 '비닐봉투(배달중)_大', '비닐봉투(배달중)_小', '비닐봉투(배달중)_특大', '사각찬용기_4칸', '샐러드_1칸(550ml)', '샐러드_3칸(150ml)', '샐러드_4칸(100ml)', '수세미', '실링기계', '실링용기_1000ml', '실링용기_1150ml', '실링용기_1350ml', '실링용기_1500ml', '실링용기_2000ml', '실링용기_200ml', '실링용기_2200ml', '실링용기_230ml', '실링용기_3-5A', '실링용기_400ml', '실링용기_500ml', '실링용기_5칸', '실링용기_600ml', '실링용기_650ml', '실링용기_700ml', '실링용기_84-2A', '실링용기_900ml', '실링필름_150mm x 200 m', '실링필름_160mm x 200m', '실링필름_190mm x 200m', '실링필름_200mm x 200m', '실링필름_245mm  x 200m', '아이스컵 뚜껑_돔형_구멍', '아이스컵 뚜껑_돔형_노구멍', '아이스컵 뚜껑_평형_민자형_십자구멍', '아이스컵_12ounce', '아이스컵_14ounce', '아이스컵_16ounce', '아이스컵_24ounce', '아이스컵_돔형_구멍', '우동용기_中', '우동용기_大', '우동용기_특大', '원형용기_150ml', '원형용기_350ml', '원형용기_450ml', '원형용기_470ml', '원형용기_大', '원형접시_大', '원형접시_小', '원형접시_미니', '위생랩_2호', '위생롤백_2호', '위생롤백_5호', '위생장갑', '일회용 숟가락', '일회용 젓가락_198mm', '일회용 포크_160mm', '종이호일_1호', '종이호일_2호',
 '죽용기_1050ml', '죽용기_1150ml', '죽용기_400ml', '죽용기_550ml', '죽용기_750ml', '죽용기_920ml', '중식소스용기', '중화면용기 뚜껑', '중화면용기_中', '중화면용기_大', '중화면용기_小', '지퍼백_M', '짬짜면_2칸', '찜용기(사각)_2000ml', '찜용기(사각)_2500ml', '찜용기(사각)_3200ml', '찜용기(원)_2500ml', '찜용기(원)_3500ml', '찜용기(원)_4000ml', '찜용기(원)_4500ml', '찜용기(타원)_1800ml', '찜용기(타원)_2500ml', '찜용기(타원)_3000ml', '찬용기_250ml', '찬용기_350ml', '찬용기_450ml', '타원접시', '탕용기_195_大', '탕용기_195_小', '탕용기_210_大', '탕용기_210_小', '탕용기_225_1800ml', '탕용기_225_2400ml', '탕용기_225_3000ml', '탕용기_240_5000ml', '탕용기_270_6500ml', '행주']]

In [11]:
mother_df = mother_df.fillna(0)

In [12]:
mother_df

,주문자ID,감자탕용기_2200ml,감자탕용기_2700ml,감자탕용기_3200ml,고강도미니탕용기_미니,고무장갑,군만두용기_1칸,냄비_2750ml,냄비뚜껑_大,냉면용기_中,냉면용기_大,다용도컵_150ml,다용도컵_165ml,다용도컵_200ml,다용도컵_250ml,다용도컵_270ml,다용도컵_300ml,다용도컵_370ml,다용도컵_400ml,다용도컵_50ml,다용도컵_600ml,다용도컵_70ml,다용도컵_80ml,단무지,도시락용기(돈까스)_5칸,도시락용기(돈까스)_中,도시락용기_4칸 A,도시락용기_4칸 B,도시락용기_5칸,도시락용기_6칸 02,도시락용기_6칸 L1,도시락용기_7칸,도시락용기_8칸,랩칼,미니탕용기_미니,반찬용기_104_1칸,반찬용기_105_1칸,반찬용기_201_1칸,반찬용기_202-2_2칸,반찬용기_202_1칸,반찬용기_203_1칸,반찬용기_204_1칸,반찬용기_211_1칸,반찬용기_D-363,반찬용기_D-364_4칸,반찬용기_D-365_5칸,비닐봉투(맛난거)_中,비닐봉투(맛난거)_大,비닐봉투(맛난거)_小,비닐봉투(맛난거)_특大,비닐봉투(무지)_中,비닐봉투(무지)_大,비닐봉투(무지)_小,비닐봉투(무지)_특大,비닐봉투(배달중)_中,비닐봉투(배달중)_大,비닐봉투(배달중)_小,비닐봉투(배달중)_특大,사각찬용기_4칸,샐러드_1칸(550ml),샐러드_3칸(150ml),샐러드_4칸(100ml),수세미,실링기계,실링용기_1000ml,실링용기_1150ml,실링용기_1350ml,실링용기_1500ml,실링용기_2000ml,실링용기_200ml,실링용기_2200ml,실링용기_230ml,실링용기_3-5A,실링용기_400ml,실링용기_500ml,실링용기_5칸,실링용기_600ml,실링용기_650ml,실링용기_700ml,실링용기_84-2A,실링용기_900ml,실링필름_150mm x 200 m,실링필름_160mm x 200m,실링필름_190mm x 200m,실링필름_200mm x 200m,실링필름_245mm x 200m,아이스컵 뚜껑_돔형_구멍,아이스컵 뚜껑_돔형_노구멍,아이스컵 뚜껑_평형_민자형_십자구멍,아이스컵_12ounce,아이스컵_14ounce,아이스컵_16ounce,아이스컵_24ounce,아이스컵_돔형_구멍,우동용기_中,우동용기_大,우동용기_특大,원형용기_150ml,원형용기_350ml,원형용기_450ml,원형용기_470ml,원형용기_大,원형접시_大,원형접시_小,원형접시_미니,위생랩_2호,위생롤백_2호,위생롤백_5호,위생장갑,일회용 숟가락,일회용 젓가락_198mm,일회용 포크_160mm,종이호일_1호,종이호일_2호,죽용기_1050ml,죽용기_1150ml,죽용기_400ml,죽용기_550ml,죽용기_750ml,죽용기_920ml,중식소스용기,중화면용기 뚜껑,중화면용기_中,중화면용기_大,중화면용기_小,지퍼백_M,짬짜면_2칸,찜용기(사각)_2000ml,찜용기(사각)_2500ml,찜용기(사각)_3200ml,찜용기(원)_2500ml,찜용기(원)_3500ml,찜용기(원)_4000ml,찜용기(원)_4500ml,찜용기(타원)_1800ml,찜용기(타원)_2500ml,찜용기(타원)_3000ml,찬용기_250ml,찬용기_350ml,찬용기_450ml,타원접시,탕용기_195_大,탕용기_195_小,탕용기_210_大,탕용기_210_小,탕용기_225_1800ml,탕용기_225_2400ml,탕용기_225_3000ml,탕용기_240_5000ml,탕용기_270_6500ml,행주
0,xigol80,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.18,0.00,0.0,0.00,0.00,0.0,70.38,0,0.0,53.97,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.20,1.3,0.0,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
0,kyen1311,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,7.99,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
0,85643404@n,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.14,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,3.55,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
0,ywca897,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,20.62,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.86,0.0,0.0,0.0,0,0,0.0,0.0,0

## father 데이터프레임 만들기
평균결제금액, 평균구매주기, 전화고객 여부, 같이 구매하는 품목

평균결제금액 column 추가

In [13]:
# 평균결제금액 column 추가
# median
father_df = pck_dropdup.groupby('주문자ID')[['총 결제금액']].median()
father_df = father_df.rename(columns = {'총 결제금액':'평균 주문금액(median)'})
father_df = father_df.reset_index()
father_df

,주문자ID,평균 주문금액(median)
0,1003409866@k,192610.0
1,100381931@n,11560.0
2,1008831959@k,99400.0
3,1009216553@k,118250.0
4,10320819@n,49700.0
...,...,...
1457,zlxm777,159965.0
1458,zuingong73,62070.0
1459,zxxz60,154650.0
1460,zzar2000,57850.0


In [14]:
# mean
list_id = []
list_order_amount_median = []
list_order_amount_mean = []
for customer_id in tqdm(list(pck_dropdup['주문자ID'].unique())):
    temp = pck_dropdup[pck_dropdup['주문자ID']==customer_id]
    order_amount_median = temp['총 결제금액'].median()
    order_amount_mean= temp['총 결제금액'].mean()
    list_id.append(customer_id)
    list_order_amount_median.append(order_amount_median)
    list_order_amount_mean.append(order_amount_mean)

100%|█████████████████████████████████████████████████████████████████████████████| 1462/1462 [00:04<00:00, 304.07it/s]


In [15]:
dict_id_mean = dict(zip(list_id, list_order_amount_mean))

father_df['평균주문금액(mean)'] = father_df['주문자ID'].map(dict_id_mean)

In [16]:
father_df

,주문자ID,평균 주문금액(median),평균주문금액(mean)
0,1003409866@k,192610.0,206449.473684
1,100381931@n,11560.0,18683.333333
2,1008831959@k,99400.0,109973.333333
3,1009216553@k,118250.0,99323.333333
4,10320819@n,49700.0,72497.222222
...,...,...,...
1457,zlxm777,159965.0,187409.583333
1458,zuingong73,62070.0,76062.000000
1459,zxxz60,154650.0,203338.333333
1460,zzar2000,57850.0,67492.500000


평균구매주기 column 추가

In [17]:
# 평균구매주기 column 추가
num = 0
customer_median_order_cycle_list = []
for customer_id in tqdm(list(pck_dropdup['주문자ID'].unique())):
    temp = pck_dropdup[pck_dropdup['주문자ID']==customer_id]
    temp = temp.sort_values(by='주문일시')
    collect_customer_order_period = []
    if (temp.shape[0]>1):
        for i in range(temp.shape[0]-1):
            period_value = temp['주문일시'].iloc[i+1] - temp['주문일시'].iloc[i]
            if period_value >  datetime.timedelta(days=1):
                collect_customer_order_period.append([customer_id,period_value])
    else:
        for i in range(temp.shape[0]):
            num= num+1
            period_value = datetime.timedelta(days=0)
            collect_customer_order_period.append([customer_id,period_value])
            print("총 {}명이 1번만 주문했지롱~그의 이름은 {}".format(num, customer_id))
            
    total = datetime.timedelta(days=0)
    order_period = []
    for i in collect_customer_order_period:
        order_period.append(i[1])
        median_order_period = (statistics.median(order_period)).days
    customer_median_order_cycle_list.append([customer_id, median_order_period])

100%|█████████████████████████████████████████████████████████████████████████████| 1462/1462 [00:07<00:00, 196.02it/s]


In [18]:
customer_id_list = []
median_order_cycle_list = []
for i in customer_median_order_cycle_list:
    customer_id_list.append(i[0])
    median_order_cycle_list.append(i[1])
dict_customer_median_order_cycle = dict(zip(customer_id_list,median_order_cycle_list))

father_df['평균구매주기'] = father_df['주문자ID'].map(dict_customer_median_order_cycle)

In [19]:
father_df

,주문자ID,평균 주문금액(median),평균주문금액(mean),평균구매주기
0,1003409866@k,192610.0,206449.473684,7
1,100381931@n,11560.0,18683.333333,51
2,1008831959@k,99400.0,109973.333333,20
3,1009216553@k,118250.0,99323.333333,39
4,10320819@n,49700.0,72497.222222,7
...,...,...,...,...
1457,zlxm777,159965.0,187409.583333,12
1458,zuingong73,62070.0,76062.000000,36
1459,zxxz60,154650.0,203338.333333,8
1460,zzar2000,57850.0,67492.500000,10


같이 구매하는 품목 column 추가

In [20]:
purchase_list = []
for customer_id in tqdm(list(pck_dropdup['주문자ID'].unique())):
    temp = pck_dropdup[pck_dropdup['주문자ID']==customer_id]
    semi_product_list = [product for product in list(set(temp['세부분류'].unique())) if type(product) is not float]  
    purchase_list.append([customer_id, semi_product_list])

100%|█████████████████████████████████████████████████████████████████████████████| 1462/1462 [00:04<00:00, 321.02it/s]


In [21]:
customer_id_list = []
product_purchase_list = []
for i in purchase_list:
    customer_id_list.append(i[0])
    product_purchase_list.append(i[1])
dict_customer_purchase_list = dict(zip(customer_id_list,product_purchase_list))

father_df['같이 사는 품목'] = father_df['주문자ID'].map(dict_customer_purchase_list)

In [22]:
father_df

,주문자ID,평균 주문금액(median),평균주문금액(mean),평균구매주기,같이 사는 품목
0,1003409866@k,192610.0,206449.473684,7,"[중화면용기, 원형접시, 우동용기]"
1,100381931@n,11560.0,18683.333333,51,[비닐봉투(무지)]
2,1008831959@k,99400.0,109973.333333,20,"[탕용기, 다용도컵, 도시락용기(돈까스)]"
3,1009216553@k,118250.0,99323.333333,39,"[다용도컵, 죽용기]"
4,10320819@n,49700.0,72497.222222,7,"[실링용기, 도시락용기(돈까스), 도시락용기]"
...,...,...,...,...,...
1457,zlxm777,159965.0,187409.583333,12,"[탕용기, 원형접시, 다용도컵, 타원접시, 짬짜면, 중화면용기]"
1458,zuingong73,62070.0,76062.000000,36,"[다용도컵, 일회용 젓가락, 비닐봉투(무지)]"
1459,zxxz60,154650.0,203338.333333,8,"[중화면용기, 원형접시, 다용도컵]"
1460,zzar2000,57850.0,67492.500000,10,"[미니탕용기, 다용도컵, 탕용기, 일회용 숟가락]"


전화주문고객 column 추가

In [23]:
father_df['전화주문고객'] =  father_df['주문자ID'].str[:2] == 'pp'

In [24]:
father_df['전화주문고객'] = father_df['전화주문고객'].replace(False,0)
father_df['전화주문고객'] = father_df['전화주문고객'].replace(True,0)

In [25]:
father_df

,주문자ID,평균 주문금액(median),평균주문금액(mean),평균구매주기,같이 사는 품목,전화주문고객
0,1003409866@k,192610.0,206449.473684,7,"[중화면용기, 원형접시, 우동용기]",0.0
1,100381931@n,11560.0,18683.333333,51,[비닐봉투(무지)],0.0
2,1008831959@k,99400.0,109973.333333,20,"[탕용기, 다용도컵, 도시락용기(돈까스)]",0.0
3,1009216553@k,118250.0,99323.333333,39,"[다용도컵, 죽용기]",0.0
4,10320819@n,49700.0,72497.222222,7,"[실링용기, 도시락용기(돈까스), 도시락용기]",0.0
...,...,...,...,...,...,...
1457,zlxm777,159965.0,187409.583333,12,"[탕용기, 원형접시, 다용도컵, 타원접시, 짬짜면, 중화면용기]",0.0
1458,zuingong73,62070.0,76062.000000,36,"[다용도컵, 일회용 젓가락, 비닐봉투(무지)]",0.0
1459,zxxz60,154650.0,203338.333333,8,"[중화면용기, 원형접시, 다용도컵]",0.0
1460,zzar2000,57850.0,67492.500000,10,"[미니탕용기, 다용도컵, 탕용기, 일회용 숟가락]",0.0


## mother_df, father_df 합치기

In [26]:
final_df = mother_df.merge(father_df, how='left', on='주문자ID')

In [27]:
# column 순서 이쁘게 정리하기
final_df = final_df[[ '주문자ID','평균 주문금액(median)','평균주문금액(mean)','평균구매주기', '같이 사는 품목','전화주문고객','감자탕용기_2200ml', '감자탕용기_2700ml', '감자탕용기_3200ml', '고강도미니탕용기_미니', '고무장갑', '군만두용기_1칸', '냄비_2750ml', '냄비뚜껑_大', '냉면용기_中', '냉면용기_大', '다용도컵_150ml', '다용도컵_165ml', '다용도컵_200ml', '다용도컵_250ml', '다용도컵_270ml', '다용도컵_300ml', '다용도컵_370ml', '다용도컵_400ml', '다용도컵_50ml', '다용도컵_600ml', '다용도컵_70ml', '다용도컵_80ml', '단무지', '도시락용기(돈까스)_5칸', '도시락용기(돈까스)_中', '도시락용기_4칸 A', '도시락용기_4칸 B', '도시락용기_5칸', '도시락용기_6칸 02', '도시락용기_6칸 L1', '도시락용기_7칸', '도시락용기_8칸', '랩칼', '미니탕용기_미니', '반찬용기_104_1칸', '반찬용기_105_1칸', '반찬용기_201_1칸', '반찬용기_202-2_2칸', '반찬용기_202_1칸', '반찬용기_203_1칸', '반찬용기_204_1칸', '반찬용기_211_1칸', '반찬용기_D-363', '반찬용기_D-364_4칸', '반찬용기_D-365_5칸', '비닐봉투(맛난거)_中', '비닐봉투(맛난거)_大', '비닐봉투(맛난거)_小', '비닐봉투(맛난거)_특大', '비닐봉투(무지)_中', '비닐봉투(무지)_大', '비닐봉투(무지)_小', '비닐봉투(무지)_특大', '비닐봉투(배달중)_中',
 '비닐봉투(배달중)_大', '비닐봉투(배달중)_小', '비닐봉투(배달중)_특大', '사각찬용기_4칸', '샐러드_1칸(550ml)', '샐러드_3칸(150ml)', '샐러드_4칸(100ml)', '수세미', '실링기계', '실링용기_1000ml', '실링용기_1150ml', '실링용기_1350ml', '실링용기_1500ml', '실링용기_2000ml', '실링용기_200ml', '실링용기_2200ml', '실링용기_230ml', '실링용기_3-5A', '실링용기_400ml', '실링용기_500ml', '실링용기_5칸', '실링용기_600ml', '실링용기_650ml', '실링용기_700ml', '실링용기_84-2A', '실링용기_900ml', '실링필름_150mm x 200 m', '실링필름_160mm x 200m', '실링필름_190mm x 200m', '실링필름_200mm x 200m', '실링필름_245mm  x 200m', '아이스컵 뚜껑_돔형_구멍', '아이스컵 뚜껑_돔형_노구멍', '아이스컵 뚜껑_평형_민자형_십자구멍', '아이스컵_12ounce', '아이스컵_14ounce', '아이스컵_16ounce', '아이스컵_24ounce', '아이스컵_돔형_구멍', '우동용기_中', '우동용기_大', '우동용기_특大', '원형용기_150ml', '원형용기_350ml', '원형용기_450ml', '원형용기_470ml', '원형용기_大', '원형접시_大', '원형접시_小', '원형접시_미니', '위생랩_2호', '위생롤백_2호', '위생롤백_5호', '위생장갑', '일회용 숟가락', '일회용 젓가락_198mm', '일회용 포크_160mm', '종이호일_1호', '종이호일_2호',
 '죽용기_1050ml', '죽용기_1150ml', '죽용기_400ml', '죽용기_550ml', '죽용기_750ml', '죽용기_920ml', '중식소스용기', '중화면용기 뚜껑', '중화면용기_中', '중화면용기_大', '중화면용기_小', '지퍼백_M', '짬짜면_2칸', '찜용기(사각)_2000ml', '찜용기(사각)_2500ml', '찜용기(사각)_3200ml', '찜용기(원)_2500ml', '찜용기(원)_3500ml', '찜용기(원)_4000ml', '찜용기(원)_4500ml', '찜용기(타원)_1800ml', '찜용기(타원)_2500ml', '찜용기(타원)_3000ml', '찬용기_250ml', '찬용기_350ml', '찬용기_450ml', '타원접시', '탕용기_195_大', '탕용기_195_小', '탕용기_210_大', '탕용기_210_小', '탕용기_225_1800ml', '탕용기_225_2400ml', '탕용기_225_3000ml', '탕용기_240_5000ml', '탕용기_270_6500ml', '행주']]

In [28]:
final_df

,주문자ID,평균 주문금액(median),평균주문금액(mean),평균구매주기,같이 사는 품목,전화주문고객,감자탕용기_2200ml,감자탕용기_2700ml,감자탕용기_3200ml,고강도미니탕용기_미니,고무장갑,군만두용기_1칸,냄비_2750ml,냄비뚜껑_大,냉면용기_中,냉면용기_大,다용도컵_150ml,다용도컵_165ml,다용도컵_200ml,다용도컵_250ml,다용도컵_270ml,다용도컵_300ml,다용도컵_370ml,다용도컵_400ml,다용도컵_50ml,다용도컵_600ml,다용도컵_70ml,다용도컵_80ml,단무지,도시락용기(돈까스)_5칸,도시락용기(돈까스)_中,도시락용기_4칸 A,도시락용기_4칸 B,도시락용기_5칸,도시락용기_6칸 02,도시락용기_6칸 L1,도시락용기_7칸,도시락용기_8칸,랩칼,미니탕용기_미니,반찬용기_104_1칸,반찬용기_105_1칸,반찬용기_201_1칸,반찬용기_202-2_2칸,반찬용기_202_1칸,반찬용기_203_1칸,반찬용기_204_1칸,반찬용기_211_1칸,반찬용기_D-363,반찬용기_D-364_4칸,반찬용기_D-365_5칸,비닐봉투(맛난거)_中,비닐봉투(맛난거)_大,비닐봉투(맛난거)_小,비닐봉투(맛난거)_특大,비닐봉투(무지)_中,비닐봉투(무지)_大,비닐봉투(무지)_小,비닐봉투(무지)_특大,비닐봉투(배달중)_中,비닐봉투(배달중)_大,비닐봉투(배달중)_小,비닐봉투(배달중)_특大,사각찬용기_4칸,샐러드_1칸(550ml),샐러드_3칸(150ml),샐러드_4칸(100ml),수세미,실링기계,실링용기_1000ml,실링용기_1150ml,실링용기_1350ml,실링용기_1500ml,실링용기_2000ml,실링용기_200ml,실링용기_2200ml,실링용기_230ml,실링용기_3-5A,실링용기_400ml,실링용기_500ml,실링용기_5칸,실링용기_600ml,실링용기_650ml,실링용기_700ml,실링용기_84-2A,실링용기_900ml,실링필름_150mm x 200 m,실링필름_160mm x 200m,실링필름_190mm x 200m,실링필름_200mm x 200m,실링필름_245mm x 200m,아이스컵 뚜껑_돔형_구멍,아이스컵 뚜껑_돔형_노구멍,아이스컵 뚜껑_평형_민자형_십자구멍,아이스컵_12ounce,아이스컵_14ounce,아이스컵_16ounce,아이스컵_24ounce,아이스컵_돔형_구멍,우동용기_中,우동용기_大,우동용기_특大,원형용기_150ml,원형용기_350ml,원형용기_450ml,원형용기_470ml,원형용기_大,원형접시_大,원형접시_小,원형접시_미니,위생랩_2호,위생롤백_2호,위생롤백_5호,위생장갑,일회용 숟가락,일회용 젓가락_198mm,일회용 포크_160mm,종이호일_1호,종이호일_2호,죽용기_1050ml,죽용기_1150ml,죽용기_400ml,죽용기_550ml,죽용기_750ml,죽용기_920ml,중식소스용기,중화면용기 뚜껑,중화면용기_中,중화면용기_大,중화면용기_小,지퍼백_M,짬짜면_2칸,찜용기(사각)_2000ml,찜용기(사각)_2500ml,찜용기(사각)_3200ml,찜용기(원)_2500ml,찜용기(원)_3500ml,찜용기(원)_4000ml,찜용기(원)_4500ml,찜용기(타원)_1800ml,찜용기(타원)_2500ml,찜용기(타원)_3000ml,찬용기_250ml,찬용기_350ml,찬용기_450ml,타원접시,탕용기_195_大,탕용기_195_小,탕용기_210_大,탕용기_210_小,탕용기_225_1800ml,탕용기_225_2400ml,탕용기_225_3000ml,탕용기_240_5000ml,탕용기_270_6500ml,행주
0,xigol80,121440.0,144826.818182,37,[다용도컵],0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.18,0.00,0.0,0.00,0.00,0.0,70.38,0,0.0,53.97,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.20,1.3,0.0,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
1,kyen1311,91620.0,89502.857143,75,[죽용기],0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,7.99,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
2,85643404@n,75820.0,73318.536585,14,"[위생장갑, 샐러드]",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.14,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,3.55,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
3,ywca897,174080.0,182370.909091,64,"[다용도컵, 죽용기]",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,20.62,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0

In [29]:
final_df.to_csv("parent_df (1462명).csv",index=False, encoding='euc-kr')

# 0. 수요예측 예시
4월까지의 data를 가지고 5월 1일 ~ 5월 14일 수요량 예측하기
Step 1) 수요예측 시점 이전에 가장 최근 주문일을 추출한다
Step 2) 이를 바탕으로 Duration day와 coefficient를 구해준다.

우선 5월 전 최근 주문일을 뽑아보기

In [30]:
pck_royal_cst

,주문번호,주문자ID,주문경로(PC/모바일),결제수단,총 결제금액,주문상품명,상품번호,주문횟수,상품명(패커스(PACKUS)),수량,분류,세부분류,사이즈,색상,개수,공급사,자체상품코드,주문일시,결제일시(입금확인일),발주일,배송대기일,배송시작일,배송완료일,세금구분,공급원가,상품 매입가,판매가,상품구매금액,상품별 추가할인 상세,옵션+판매가,쿠폰 할인금액,주문자 가입일,특별관리회원,회원추가항목_가입경로,회원추가항목_업종,주문 시 회원등급,주문자 주소,수령인 주소,위도,경도,주문-배송완료,발주일-배송시작일,배송시작-배송완료,회원가입연도,회원가입월,회원가입일,가장최근주문,19년이후 누적주문횟수,가입기간(일수),누적주문,주문연도.월,총주문량,세부분류사이즈
0,20170323-0000017,xigol80,PC,"쿠폰,카드",169500,"70파이 다용도컵 세트 1호 소 70ml 백색 PS 3,000개 1set(2박스)",38,477.0,"70파이 다용도컵 세트 1호 소 70ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,70ml,백색,6000.0,패커스,P00000BL,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 10:32:06.096,2017-03-23 18:43:40,2017-04-01 19:49:03,과세상품,42900.0,40500,53100,53100.0,NaN,53100.0,2000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,9 days 16:39:45.000000000,0 days 15:34:22.000000000,9 days 01:05:23.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_3,6000.0,다용도컵_70ml
1,20170323-0000017,xigol80,PC,"쿠폰,카드",169500,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42,471.0,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",2,30.0,다용도컵,200ml,백색,2000.0,패커스,P00000BP,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 10:33:14.195,2017-03-23 18:43:40,2017-04-01 19:49:03,과세상품,44000.0,43000,59200,118400.0,NaN,59200.0,2000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,9 days 16:39:45.000000000,0 days 15:34:22.000000000,9 days 01:05:23.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_3,4000.0,다용도컵_200ml
2,20170421-0000015,xigol80,PC,"쿠폰,카드",113400,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42,471.0,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",2,30.0,다용도컵,200ml,백색,2000.0,패커스,P00000BP,2017-04-21 03:38:14,2017-04-21 03:38:14,2017-04-21 03:38:14,2017-04-21 17:05:22.809,2017-04-21 17:10:58,2017-04-25 08:23:56,과세상품,44000.0,43000,59200,118400.0,NaN,59200.0,5000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,4 days 04:45:42.000000000,0 days 13:32:44.000000000,3 days 15:12:58.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_4,4000.0,다용도컵_200ml
3,20170519-0000012,xigol80,PC,"쿠폰,카드",208590,"70파이 다용도컵 세트 1호 특소 50ml 백색 PS 3,000개 1set(2박스)",47,274.0,"70파이 다용도컵 세트 1호 특소 50ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,50ml,백색,6000.0,패커스,P00000BU,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 16:01:20.590,2017-05-19 19:50:23,2017-05-27 05:35:19,과세상품,41250.0,40500,53100,53100.0,다용도 10%할인,53100.0,3000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,7 days 23:35:52.000000000,0 days 13:50:56.000000000,7 days 09:44:56.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_5,6000.0,다용도컵_50ml
4,20170519-0000012,xigol80,PC,"쿠폰,카드",208590,"70파이 다용도컵 세트 2호 대 80ml 백색 PS 3,000개 1set(2박스)",39,94.0,"70파이 다용도컵 세트 2호 대 80ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,80ml,백색,6000.0,패커스,P00000BM,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 16:01:20.590,2017-05-19 19:50:23,2017-05-27 05:35:19,과세상품,49500.0,45000,63600,63600.0,다용도 10%할인,63600.0,3000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,7 days 23:35:52.000000000,0 days 13:50:56.000000000,7 days 09:44:56.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_5,6000.0,다용도컵_80ml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33199,1908301422425566,q9691445,NaN,무통장 입금,54850,195파이 탕용기 소 300개 1set(2박스),1077,1366.0,195파이 탕용기 소 300개 1set(2박스),1,1.0,탕용기,195_小,NaN,600.0,패커스,P0000BPK,2019-08-30 14:22:42,2019-09-02 09:03:36,NaT,NaT,NaT,NaT,NaN,NaN,44550,57850,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,새내기,NaN,경상북도 포항시 북구 성실로 21-1,36.065835,129.376172,NaN,NaN,NaN,NaT,NaT,NaT,2019-08-30 14:22:42,20.0,1004,11.0,2019_8,600.0,탕용기_195_小
33200,1908301139376461,pp039,NaN,무통장 입금,155

In [31]:
list_customer_id = []
list_timestamp = []
for customer_id in tqdm(list(pck_royal_cst['주문자ID'].unique())):
    temp = pck_royal_cst_dropdup[pck_royal_cst_dropdup['주문자ID']==customer_id]
    timestamp = datetime.datetime(year=2019, month=5, day=1)
    for num in range(temp.shape[0]-1,-1,-1):
        order_before_may = temp['주문일시'].iloc[num]
        if (order_before_may < timestamp):
            list_timestamp.append(order_before_may)
            list_customer_id.append(customer_id)
            break

100%|█████████████████████████████████████████████████████████████████████████████| 1462/1462 [00:04<00:00, 345.13it/s]


In [32]:
dict_customerid_timestamp = dict(zip(list_customer_id,list_timestamp))
dict_customerid_timestamp

{'xigol80': Timestamp('2019-04-02 03:40:35'),
 'kyen1311': Timestamp('2019-03-24 23:37:51'),
 '85643404@n': Timestamp('2019-04-23 11:00:30'),
 'ywca897': Timestamp('2019-04-08 17:59:22'),
 'pinkbabi100': Timestamp('2019-04-02 13:04:45'),
 'yyk1104': Timestamp('2019-03-11 14:30:16'),
 'rudals730': Timestamp('2019-03-21 14:24:18'),
 '39618168@n': Timestamp('2019-04-03 20:20:09'),
 'duruzoon70': Timestamp('2019-04-30 15:31:58'),
 'dhjekd': Timestamp('2019-04-29 16:44:28'),
 'jjangbeom': Timestamp('2019-04-21 17:36:45'),
 'rlawltnr85': Timestamp('2019-04-11 13:19:04'),
 'kkh5652': Timestamp('2019-02-07 12:48:17'),
 'sunga0121': Timestamp('2019-04-22 21:06:41'),
 'love1203a': Timestamp('2019-01-09 08:52:01'),
 'ilmifood': Timestamp('2019-03-27 17:50:32'),
 'sarew81': Timestamp('2018-08-07 17:39:41'),
 'noble602': Timestamp('2019-04-03 11:03:16'),
 '5664820@n': Timestamp('2019-01-28 22:53:42'),
 'mission1013': Timestamp('2019-04-27 14:45:31'),
 'ksj8040': Timestamp('2019-04-30 09:53:59'),
 '

In [33]:
pck_royal_cst['5월전 마지막 주문'] = pck_royal_cst['주문자ID'].map(dict_customerid_timestamp)

In [34]:
pck_royal_cst

,주문번호,주문자ID,주문경로(PC/모바일),결제수단,총 결제금액,주문상품명,상품번호,주문횟수,상품명(패커스(PACKUS)),수량,분류,세부분류,사이즈,색상,개수,공급사,자체상품코드,주문일시,결제일시(입금확인일),발주일,배송대기일,배송시작일,배송완료일,세금구분,공급원가,상품 매입가,판매가,상품구매금액,상품별 추가할인 상세,옵션+판매가,쿠폰 할인금액,주문자 가입일,특별관리회원,회원추가항목_가입경로,회원추가항목_업종,주문 시 회원등급,주문자 주소,수령인 주소,위도,경도,주문-배송완료,발주일-배송시작일,배송시작-배송완료,회원가입연도,회원가입월,회원가입일,가장최근주문,19년이후 누적주문횟수,가입기간(일수),누적주문,주문연도.월,총주문량,세부분류사이즈,5월전 마지막 주문
0,20170323-0000017,xigol80,PC,"쿠폰,카드",169500,"70파이 다용도컵 세트 1호 소 70ml 백색 PS 3,000개 1set(2박스)",38,477.0,"70파이 다용도컵 세트 1호 소 70ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,70ml,백색,6000.0,패커스,P00000BL,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 10:32:06.096,2017-03-23 18:43:40,2017-04-01 19:49:03,과세상품,42900.0,40500,53100,53100.0,NaN,53100.0,2000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,9 days 16:39:45.000000000,0 days 15:34:22.000000000,9 days 01:05:23.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_3,6000.0,다용도컵_70ml,2019-04-02 03:40:35
1,20170323-0000017,xigol80,PC,"쿠폰,카드",169500,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42,471.0,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",2,30.0,다용도컵,200ml,백색,2000.0,패커스,P00000BP,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 03:09:18,2017-03-23 10:33:14.195,2017-03-23 18:43:40,2017-04-01 19:49:03,과세상품,44000.0,43000,59200,118400.0,NaN,59200.0,2000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,9 days 16:39:45.000000000,0 days 15:34:22.000000000,9 days 01:05:23.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_3,4000.0,다용도컵_200ml,2019-04-02 03:40:35
2,20170421-0000015,xigol80,PC,"쿠폰,카드",113400,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42,471.0,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",2,30.0,다용도컵,200ml,백색,2000.0,패커스,P00000BP,2017-04-21 03:38:14,2017-04-21 03:38:14,2017-04-21 03:38:14,2017-04-21 17:05:22.809,2017-04-21 17:10:58,2017-04-25 08:23:56,과세상품,44000.0,43000,59200,118400.0,NaN,59200.0,5000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,4 days 04:45:42.000000000,0 days 13:32:44.000000000,3 days 15:12:58.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_4,4000.0,다용도컵_200ml,2019-04-02 03:40:35
3,20170519-0000012,xigol80,PC,"쿠폰,카드",208590,"70파이 다용도컵 세트 1호 특소 50ml 백색 PS 3,000개 1set(2박스)",47,274.0,"70파이 다용도컵 세트 1호 특소 50ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,50ml,백색,6000.0,패커스,P00000BU,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 16:01:20.590,2017-05-19 19:50:23,2017-05-27 05:35:19,과세상품,41250.0,40500,53100,53100.0,다용도 10%할인,53100.0,3000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,7 days 23:35:52.000000000,0 days 13:50:56.000000000,7 days 09:44:56.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_5,6000.0,다용도컵_50ml,2019-04-02 03:40:35
4,20170519-0000012,xigol80,PC,"쿠폰,카드",208590,"70파이 다용도컵 세트 2호 대 80ml 백색 PS 3,000개 1set(2박스)",39,94.0,"70파이 다용도컵 세트 2호 대 80ml 백색 PS 3,000개 1set(2박스)",1,30.0,다용도컵,80ml,백색,6000.0,패커스,P00000BM,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 05:59:27,2017-05-19 16:01:20.590,2017-05-19 19:50:23,2017-05-27 05:35:19,과세상품,49500.0,45000,63600,63600.0,다용도 10%할인,63600.0,3000.0,2017-03-08,NaN,NaN,NaN,사업자,서울특별시 영등포구 여의도동,서울특별시 영등포구 여의도동,37.517540,126.934531,7 days 23:35:52.000000000,0 days 13:50:56.000000000,7 days 09:44:56.000000000,2017,3,3,2019-05-03 04:09:24,51.0,1004,22.0,2017_5,6000.0,다용도컵_80ml,2019-04-02 03:40:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33199,1908301422425566,q9691445,NaN,무통장 입금,54850,195파이 탕용기 소 300개 1set(2박스),1077,1366.0,195파이 탕용기 소 300개 1set(2박스),1,1.0,탕용기,195_小,NaN,600.0,패커스,P0000BPK,2019-08-30 14:22:42,2019-09-02 09:03:36,NaT,NaT,NaT,NaT,NaN,NaN,44550,57850,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,새내기,NaN,경상북도 포항시 북구 성실로 21-1,36.065835,129.376172,NaN,NaN,Na

4월 재고 예측하기

In [4]:
data = pd.read_csv("data/parent_df (1462명).csv", encoding='euc-kr')

In [36]:
data

,주문자ID,평균 주문금액(median),평균주문금액(mean),평균구매주기,같이 사는 품목,전화주문고객,감자탕용기_2200ml,감자탕용기_2700ml,감자탕용기_3200ml,고강도미니탕용기_미니,고무장갑,군만두용기_1칸,냄비_2750ml,냄비뚜껑_大,냉면용기_中,냉면용기_大,다용도컵_150ml,다용도컵_165ml,다용도컵_200ml,다용도컵_250ml,다용도컵_270ml,다용도컵_300ml,다용도컵_370ml,다용도컵_400ml,다용도컵_50ml,다용도컵_600ml,다용도컵_70ml,다용도컵_80ml,단무지,도시락용기(돈까스)_5칸,도시락용기(돈까스)_中,도시락용기_4칸 A,도시락용기_4칸 B,도시락용기_5칸,도시락용기_6칸 02,도시락용기_6칸 L1,도시락용기_7칸,도시락용기_8칸,랩칼,미니탕용기_미니,반찬용기_104_1칸,반찬용기_105_1칸,반찬용기_201_1칸,반찬용기_202-2_2칸,반찬용기_202_1칸,반찬용기_203_1칸,반찬용기_204_1칸,반찬용기_211_1칸,반찬용기_D-363,반찬용기_D-364_4칸,반찬용기_D-365_5칸,비닐봉투(맛난거)_中,비닐봉투(맛난거)_大,비닐봉투(맛난거)_小,비닐봉투(맛난거)_특大,비닐봉투(무지)_中,비닐봉투(무지)_大,비닐봉투(무지)_小,비닐봉투(무지)_특大,비닐봉투(배달중)_中,비닐봉투(배달중)_大,비닐봉투(배달중)_小,비닐봉투(배달중)_특大,사각찬용기_4칸,샐러드_1칸(550ml),샐러드_3칸(150ml),샐러드_4칸(100ml),수세미,실링기계,실링용기_1000ml,실링용기_1150ml,실링용기_1350ml,실링용기_1500ml,실링용기_2000ml,실링용기_200ml,실링용기_2200ml,실링용기_230ml,실링용기_3-5A,실링용기_400ml,실링용기_500ml,실링용기_5칸,실링용기_600ml,실링용기_650ml,실링용기_700ml,실링용기_84-2A,실링용기_900ml,실링필름_150mm x 200 m,실링필름_160mm x 200m,실링필름_190mm x 200m,실링필름_200mm x 200m,실링필름_245mm x 200m,아이스컵 뚜껑_돔형_구멍,아이스컵 뚜껑_돔형_노구멍,아이스컵 뚜껑_평형_민자형_십자구멍,아이스컵_12ounce,아이스컵_14ounce,아이스컵_16ounce,아이스컵_24ounce,아이스컵_돔형_구멍,우동용기_中,우동용기_大,우동용기_특大,원형용기_150ml,원형용기_350ml,원형용기_450ml,원형용기_470ml,원형용기_大,원형접시_大,원형접시_小,원형접시_미니,위생랩_2호,위생롤백_2호,위생롤백_5호,위생장갑,일회용 숟가락,일회용 젓가락_198mm,일회용 포크_160mm,종이호일_1호,종이호일_2호,죽용기_1050ml,죽용기_1150ml,죽용기_400ml,죽용기_550ml,죽용기_750ml,죽용기_920ml,중식소스용기,중화면용기 뚜껑,중화면용기_中,중화면용기_大,중화면용기_小,지퍼백_M,짬짜면_2칸,찜용기(사각)_2000ml,찜용기(사각)_2500ml,찜용기(사각)_3200ml,찜용기(원)_2500ml,찜용기(원)_3500ml,찜용기(원)_4000ml,찜용기(원)_4500ml,찜용기(타원)_1800ml,찜용기(타원)_2500ml,찜용기(타원)_3000ml,찬용기_250ml,찬용기_350ml,찬용기_450ml,타원접시,탕용기_195_大,탕용기_195_小,탕용기_210_大,탕용기_210_小,탕용기_225_1800ml,탕용기_225_2400ml,탕용기_225_3000ml,탕용기_240_5000ml,탕용기_270_6500ml,행주
0,xigol80,121440.0,144826.818182,37,['다용도컵'],0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.18,0.00,0.0,0.00,0.00,0.0,70.38,0,0.0,53.97,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.20,1.3,0.0,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
1,kyen1311,91620.0,89502.857143,75,['죽용기'],0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,7.99,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
2,85643404@n,75820.0,73318.536585,14,"['위생장갑', '샐러드']",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.14,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,3.55,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00
3,ywca897,174080.0,182370.909091,64,"['다용도컵', '죽용기']",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,20.62,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [37]:
data['5월전마지막주문'] = data['주문자ID'].map(dict_customerid_timestamp)

In [38]:
data

,주문자ID,평균 주문금액(median),평균주문금액(mean),평균구매주기,같이 사는 품목,전화주문고객,감자탕용기_2200ml,감자탕용기_2700ml,감자탕용기_3200ml,고강도미니탕용기_미니,고무장갑,군만두용기_1칸,냄비_2750ml,냄비뚜껑_大,냉면용기_中,냉면용기_大,다용도컵_150ml,다용도컵_165ml,다용도컵_200ml,다용도컵_250ml,다용도컵_270ml,다용도컵_300ml,다용도컵_370ml,다용도컵_400ml,다용도컵_50ml,다용도컵_600ml,다용도컵_70ml,다용도컵_80ml,단무지,도시락용기(돈까스)_5칸,도시락용기(돈까스)_中,도시락용기_4칸 A,도시락용기_4칸 B,도시락용기_5칸,도시락용기_6칸 02,도시락용기_6칸 L1,도시락용기_7칸,도시락용기_8칸,랩칼,미니탕용기_미니,반찬용기_104_1칸,반찬용기_105_1칸,반찬용기_201_1칸,반찬용기_202-2_2칸,반찬용기_202_1칸,반찬용기_203_1칸,반찬용기_204_1칸,반찬용기_211_1칸,반찬용기_D-363,반찬용기_D-364_4칸,반찬용기_D-365_5칸,비닐봉투(맛난거)_中,비닐봉투(맛난거)_大,비닐봉투(맛난거)_小,비닐봉투(맛난거)_특大,비닐봉투(무지)_中,비닐봉투(무지)_大,비닐봉투(무지)_小,비닐봉투(무지)_특大,비닐봉투(배달중)_中,비닐봉투(배달중)_大,비닐봉투(배달중)_小,비닐봉투(배달중)_특大,사각찬용기_4칸,샐러드_1칸(550ml),샐러드_3칸(150ml),샐러드_4칸(100ml),수세미,실링기계,실링용기_1000ml,실링용기_1150ml,실링용기_1350ml,실링용기_1500ml,실링용기_2000ml,실링용기_200ml,실링용기_2200ml,실링용기_230ml,실링용기_3-5A,실링용기_400ml,실링용기_500ml,실링용기_5칸,실링용기_600ml,실링용기_650ml,실링용기_700ml,실링용기_84-2A,실링용기_900ml,실링필름_150mm x 200 m,실링필름_160mm x 200m,실링필름_190mm x 200m,실링필름_200mm x 200m,실링필름_245mm x 200m,아이스컵 뚜껑_돔형_구멍,아이스컵 뚜껑_돔형_노구멍,아이스컵 뚜껑_평형_민자형_십자구멍,아이스컵_12ounce,아이스컵_14ounce,아이스컵_16ounce,아이스컵_24ounce,아이스컵_돔형_구멍,우동용기_中,우동용기_大,우동용기_특大,원형용기_150ml,원형용기_350ml,원형용기_450ml,원형용기_470ml,원형용기_大,원형접시_大,원형접시_小,원형접시_미니,위생랩_2호,위생롤백_2호,위생롤백_5호,위생장갑,일회용 숟가락,일회용 젓가락_198mm,일회용 포크_160mm,종이호일_1호,종이호일_2호,죽용기_1050ml,죽용기_1150ml,죽용기_400ml,죽용기_550ml,죽용기_750ml,죽용기_920ml,중식소스용기,중화면용기 뚜껑,중화면용기_中,중화면용기_大,중화면용기_小,지퍼백_M,짬짜면_2칸,찜용기(사각)_2000ml,찜용기(사각)_2500ml,찜용기(사각)_3200ml,찜용기(원)_2500ml,찜용기(원)_3500ml,찜용기(원)_4000ml,찜용기(원)_4500ml,찜용기(타원)_1800ml,찜용기(타원)_2500ml,찜용기(타원)_3000ml,찬용기_250ml,찬용기_350ml,찬용기_450ml,타원접시,탕용기_195_大,탕용기_195_小,탕용기_210_大,탕용기_210_小,탕용기_225_1800ml,탕용기_225_2400ml,탕용기_225_3000ml,탕용기_240_5000ml,탕용기_270_6500ml,행주,5월전마지막주문
0,xigol80,121440.0,144826.818182,37,['다용도컵'],0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.18,0.00,0.0,0.00,0.00,0.0,70.38,0,0.0,53.97,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.20,1.3,0.0,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,2019-04-02 03:40:35
1,kyen1311,91620.0,89502.857143,75,['죽용기'],0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,0.0,0,0,0.0,0.0,0.0,7.99,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,2019-03-24 23:37:51
2,85643404@n,75820.0,73318.536585,14,"['위생장갑', '샐러드']",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.14,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,3.55,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,2019-04-23 11:00:30
3,ywca897,174080.0,182370.909091,64,"['다용도컵', '죽용기']",0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,20.62,0.0,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

5월 전 주문을 했던 사람들 모으기

In [39]:
data.shape

(1462, 157)

In [40]:
temp = data[data['5월전마지막주문'].notnull()]

In [41]:
temp.shape

(1448, 157)

고객의 마지막 주문일부터 5월 14일까지의 Duration day 구해주기

In [42]:
present_time = datetime.datetime(year=2019, month=5, day=1)
stock_predict_period = datetime.timedelta(days=14)

temp['Duration_day'] = (present_time + stock_predict_period - data['5월전마지막주문'])

for i in tqdm(range(temp.shape[0])):
    temp['Duration_day'].iloc[i] = temp['Duration_day'].iloc[i].days

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
  0%|                                                                                         | 0/1448 [00:00<?, ?it/s]c:\programdata\miniconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [43]:
temp

,주문자ID,평균 주문금액(median),평균주문금액(mean),평균구매주기,같이 사는 품목,전화주문고객,감자탕용기_2200ml,감자탕용기_2700ml,감자탕용기_3200ml,고강도미니탕용기_미니,고무장갑,군만두용기_1칸,냄비_2750ml,냄비뚜껑_大,냉면용기_中,냉면용기_大,다용도컵_150ml,다용도컵_165ml,다용도컵_200ml,다용도컵_250ml,다용도컵_270ml,다용도컵_300ml,다용도컵_370ml,다용도컵_400ml,다용도컵_50ml,다용도컵_600ml,다용도컵_70ml,다용도컵_80ml,단무지,도시락용기(돈까스)_5칸,도시락용기(돈까스)_中,도시락용기_4칸 A,도시락용기_4칸 B,도시락용기_5칸,도시락용기_6칸 02,도시락용기_6칸 L1,도시락용기_7칸,도시락용기_8칸,랩칼,미니탕용기_미니,반찬용기_104_1칸,반찬용기_105_1칸,반찬용기_201_1칸,반찬용기_202-2_2칸,반찬용기_202_1칸,반찬용기_203_1칸,반찬용기_204_1칸,반찬용기_211_1칸,반찬용기_D-363,반찬용기_D-364_4칸,반찬용기_D-365_5칸,비닐봉투(맛난거)_中,비닐봉투(맛난거)_大,비닐봉투(맛난거)_小,비닐봉투(맛난거)_특大,비닐봉투(무지)_中,비닐봉투(무지)_大,비닐봉투(무지)_小,비닐봉투(무지)_특大,비닐봉투(배달중)_中,비닐봉투(배달중)_大,비닐봉투(배달중)_小,비닐봉투(배달중)_특大,사각찬용기_4칸,샐러드_1칸(550ml),샐러드_3칸(150ml),샐러드_4칸(100ml),수세미,실링기계,실링용기_1000ml,실링용기_1150ml,실링용기_1350ml,실링용기_1500ml,실링용기_2000ml,실링용기_200ml,실링용기_2200ml,실링용기_230ml,실링용기_3-5A,실링용기_400ml,실링용기_500ml,실링용기_5칸,실링용기_600ml,실링용기_650ml,실링용기_700ml,실링용기_84-2A,실링용기_900ml,실링필름_150mm x 200 m,실링필름_160mm x 200m,실링필름_190mm x 200m,실링필름_200mm x 200m,실링필름_245mm x 200m,아이스컵 뚜껑_돔형_구멍,아이스컵 뚜껑_돔형_노구멍,아이스컵 뚜껑_평형_민자형_십자구멍,아이스컵_12ounce,아이스컵_14ounce,아이스컵_16ounce,아이스컵_24ounce,아이스컵_돔형_구멍,우동용기_中,우동용기_大,우동용기_특大,원형용기_150ml,원형용기_350ml,원형용기_450ml,원형용기_470ml,원형용기_大,원형접시_大,원형접시_小,원형접시_미니,위생랩_2호,위생롤백_2호,위생롤백_5호,위생장갑,일회용 숟가락,일회용 젓가락_198mm,일회용 포크_160mm,종이호일_1호,종이호일_2호,죽용기_1050ml,죽용기_1150ml,죽용기_400ml,죽용기_550ml,죽용기_750ml,죽용기_920ml,중식소스용기,중화면용기 뚜껑,중화면용기_中,중화면용기_大,중화면용기_小,지퍼백_M,짬짜면_2칸,찜용기(사각)_2000ml,찜용기(사각)_2500ml,찜용기(사각)_3200ml,찜용기(원)_2500ml,찜용기(원)_3500ml,찜용기(원)_4000ml,찜용기(원)_4500ml,찜용기(타원)_1800ml,찜용기(타원)_2500ml,찜용기(타원)_3000ml,찬용기_250ml,찬용기_350ml,찬용기_450ml,타원접시,탕용기_195_大,탕용기_195_小,탕용기_210_大,탕용기_210_小,탕용기_225_1800ml,탕용기_225_2400ml,탕용기_225_3000ml,탕용기_240_5000ml,탕용기_270_6500ml,행주,5월전마지막주문,Duration_day
0,xigol80,121440.0,144826.818182,37,['다용도컵'],0.0,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,0.0,91.18,0.00,0.0,0.00,0.00,0.00,70.38,0,0.0,53.97,0.0,0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0,0,0,1.20,1.30,0.00,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2019-04-02 03:40:35,42
1,kyen1311,91620.0,89502.857143,75,['죽용기'],0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,0.0,0,0,0.0,0.0,0.0,7.99,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2019-03-24 23:37:51,51
2,85643404@n,75820.0,73318.536585,14,"['위생장갑', '샐러드']",0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.14,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0,0,0,3.55,0.00,0.00,0.0,0,0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2019-04-23 11:00:30,21
3,ywca897,174080.0,182370.909091,64,"['다용도컵', '죽용기']",0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,20.62,0.00,0.00,0,0.0,0.00,0.0,0,0.0,0.0,0.0,0.00,0.0,0.0,0.0

계수(coefficient) 구해주기

In [44]:
temp['coefficient'] = temp['Duration_day'] / temp['평균구매주기']
for i in tqdm(range(temp.shape[0])):
    if temp['coefficient'].iloc[i] > 1 :
        temp['coefficient'].iloc[i] = 1        

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
  0%|                                                                                         | 0/1448 [00:00<?, ?it/s]c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
100%|████████████████████████████████████████████████████████████████████████████| 1448/1448 [00:00<00:00, 3747.29it/s]


평균구매주기가 60일 이상인 고객들은 제외

In [45]:
for_coefficient = temp[temp['평균구매주기']<60]

In [46]:
# column 순서 이쁘게 정리하기
stock_predict = for_coefficient[['감자탕용기_2200ml', '감자탕용기_2700ml', '감자탕용기_3200ml', '고강도미니탕용기_미니', '고무장갑', '군만두용기_1칸', '냄비_2750ml', '냄비뚜껑_大', '냉면용기_中', '냉면용기_大', '다용도컵_150ml', '다용도컵_165ml', '다용도컵_200ml', '다용도컵_250ml', '다용도컵_270ml', '다용도컵_300ml', '다용도컵_370ml', '다용도컵_400ml', '다용도컵_50ml', '다용도컵_600ml', '다용도컵_70ml', '다용도컵_80ml', '단무지', '도시락용기(돈까스)_5칸', '도시락용기(돈까스)_中', '도시락용기_4칸 A', '도시락용기_4칸 B', '도시락용기_5칸', '도시락용기_6칸 02', '도시락용기_6칸 L1', '도시락용기_7칸', '도시락용기_8칸', '랩칼', '미니탕용기_미니', '반찬용기_104_1칸', '반찬용기_105_1칸', '반찬용기_201_1칸', '반찬용기_202-2_2칸', '반찬용기_202_1칸', '반찬용기_203_1칸', '반찬용기_204_1칸', '반찬용기_211_1칸', '반찬용기_D-363', '반찬용기_D-364_4칸', '반찬용기_D-365_5칸', '비닐봉투(맛난거)_中', '비닐봉투(맛난거)_大', '비닐봉투(맛난거)_小', '비닐봉투(맛난거)_특大', '비닐봉투(무지)_中', '비닐봉투(무지)_大', '비닐봉투(무지)_小', '비닐봉투(무지)_특大', '비닐봉투(배달중)_中',
 '비닐봉투(배달중)_大', '비닐봉투(배달중)_小', '비닐봉투(배달중)_특大', '사각찬용기_4칸', '샐러드_1칸(550ml)', '샐러드_3칸(150ml)', '샐러드_4칸(100ml)', '수세미', '실링기계', '실링용기_1000ml', '실링용기_1150ml', '실링용기_1350ml', '실링용기_1500ml', '실링용기_2000ml', '실링용기_200ml', '실링용기_2200ml', '실링용기_230ml', '실링용기_3-5A', '실링용기_400ml', '실링용기_500ml', '실링용기_5칸', '실링용기_600ml', '실링용기_650ml', '실링용기_700ml', '실링용기_84-2A', '실링용기_900ml', '실링필름_150mm x 200 m', '실링필름_160mm x 200m', '실링필름_190mm x 200m', '실링필름_200mm x 200m', '실링필름_245mm  x 200m', '아이스컵 뚜껑_돔형_구멍', '아이스컵 뚜껑_돔형_노구멍', '아이스컵 뚜껑_평형_민자형_십자구멍', '아이스컵_12ounce', '아이스컵_14ounce', '아이스컵_16ounce', '아이스컵_24ounce', '아이스컵_돔형_구멍', '우동용기_中', '우동용기_大', '우동용기_특大', '원형용기_150ml', '원형용기_350ml', '원형용기_450ml', '원형용기_470ml', '원형용기_大', '원형접시_大', '원형접시_小', '원형접시_미니', '위생랩_2호', '위생롤백_2호', '위생롤백_5호', '위생장갑', '일회용 숟가락', '일회용 젓가락_198mm', '일회용 포크_160mm', '종이호일_1호', '종이호일_2호',
 '죽용기_1050ml', '죽용기_1150ml', '죽용기_400ml', '죽용기_550ml', '죽용기_750ml', '죽용기_920ml', '중식소스용기', '중화면용기 뚜껑', '중화면용기_中', '중화면용기_大', '중화면용기_小', '지퍼백_M', '짬짜면_2칸', '찜용기(사각)_2000ml', '찜용기(사각)_2500ml', '찜용기(사각)_3200ml', '찜용기(원)_2500ml', '찜용기(원)_3500ml', '찜용기(원)_4000ml', '찜용기(원)_4500ml', '찜용기(타원)_1800ml', '찜용기(타원)_2500ml', '찜용기(타원)_3000ml', '찬용기_250ml', '찬용기_350ml', '찬용기_450ml', '타원접시', '탕용기_195_大', '탕용기_195_小', '탕용기_210_大', '탕용기_210_小', '탕용기_225_1800ml', '탕용기_225_2400ml', '탕용기_225_3000ml', '탕용기_240_5000ml', '탕용기_270_6500ml', '행주']]

In [47]:
list_coefficient = for_coefficient['coefficient'].tolist()

stock_predict = stock_predict.mul(list_coefficient, axis = 0)
stock_predict_T = stock_predict.mul(list_coefficient, axis = 0).T

In [48]:
stock_predict_T['sum'] = stock_predict_T.sum(axis=1)

In [49]:
predict_2019_5 = stock_predict_T['sum']*14

In [50]:
predict_2019_5

감자탕용기_2200ml             4191.035283
감자탕용기_2700ml              729.659914
감자탕용기_3200ml              704.480000
고강도미니탕용기_미니              2711.484524
고무장갑                       79.057874
군만두용기_1칸                 1582.455938
냄비_2750ml                 166.740000
냄비뚜껑_大                    166.740000
냉면용기_中                   2472.606289
냉면용기_大                   1681.270431
다용도컵_150ml              16866.530097
다용도컵_165ml              11037.839846
다용도컵_200ml              34211.357825
다용도컵_250ml              62297.361680
다용도컵_270ml              29067.374766
다용도컵_300ml              70175.264797
다용도컵_370ml              92843.695355
다용도컵_400ml              15738.279859
다용도컵_50ml               29146.455203
다용도컵_600ml                  0.000000
다용도컵_70ml              115524.810670
다용도컵_80ml               11205.683644
단무지                     23742.373057
도시락용기(돈까스)_5칸               0.000000
도시락용기(돈까스)_中            21027.922549
도시락용기_4칸 A               2023.061819
도시락용기_4칸 B               3572.051699
도

In [51]:
dict_predict_2019_5 = dict(zip(predict_2019_5.index, predict_2019_5.values))

5월 둘째주까지 실제 주문량과 비교

In [52]:
may_01 = datetime.datetime(year=2019, month=5, day=1)
may_14 = datetime.datetime(year=2019, month=5, day=14)
order_may = pck_royal_cst[(pck_royal_cst['주문일시']<=may_14) &(pck_royal_cst['주문일시']>may_01)]

In [53]:
order_may['주문자ID'].nunique()

541

In [54]:
list_item = []
list_item_total_order = []
for item in list(order_may['세부분류사이즈'].unique()):
    temp = order_may[order_may['세부분류사이즈']==item]
    item_total_order = temp['총주문량'].sum()
    list_item.append(item)
    list_item_total_order.append(item_total_order)

In [55]:
data = {'상품명': list_item,'총합':list_item_total_order}
real_order_amount_2019_05 = pd.DataFrame(data,columns=['상품명','총합'])
real_order_amount_2019_05 = real_order_amount_2019_05.sort_values(by='상품명')

In [56]:
real_order_amount_2019_05['예측량'] = real_order_amount_2019_05['상품명'].map(dict_predict_2019_5)

In [57]:
real_order_amount_2019_05

,상품명,총합,예측량
72,감자탕용기_2200ml,3600.0,4191.035283
105,감자탕용기_2700ml,1200.0,729.659914
108,감자탕용기_3200ml,800.0,704.480000
115,고강도미니탕용기_미니,800.0,2711.484524
113,군만두용기_1칸,1000.0,1582.455938
57,냉면용기_中,4800.0,2472.606289
116,냉면용기_大,800.0,1681.270431
29,다용도컵_150ml,12000.0,16866.530097
59,다용도컵_165ml,12000.0,11037.839846
12,다용도컵_200ml,36000.0,34211.357825


In [58]:
real_order_amount_2019_05['비율'] = real_order_amount_2019_05['예측량'] / real_order_amount_2019_05['총합']
df_top15 = real_order_amount_2019_05.sort_values(by='총합', ascending=False).head(15)

In [59]:
df_top15

,상품명,총합,예측량,비율
8,다용도컵_70ml,78000.0,115524.810670,1.481087
2,일회용 숟가락,72000.0,116348.197608,1.615947
37,다용도컵_250ml,62000.0,62297.361680,1.004796
3,다용도컵_370ml,56000.0,92843.695355,1.657923
7,다용도컵_300ml,52000.0,70175.264797,1.349524
14,미니탕용기_미니,49800.0,60336.933827,1.211585
20,탕용기_195_小,43800.0,44413.524182,1.014007
4,중화면용기_小,36600.0,63750.929324,1.741829
52,원형용기_150ml,36000.0,36471.246173,1.013090
13,탕용기_195_大,36000.0,35611.523028,0.989209


In [68]:
df_top15['비율'].median()

1.2144356334552093

In [61]:
real_order_amount_2019_05.sort_values(by='총합', ascending=False)

,상품명,총합,예측량,비율
8,다용도컵_70ml,78000.0,115524.810670,1.481087
2,일회용 숟가락,72000.0,116348.197608,1.615947
37,다용도컵_250ml,62000.0,62297.361680,1.004796
3,다용도컵_370ml,56000.0,92843.695355,1.657923
7,다용도컵_300ml,52000.0,70175.264797,1.349524
14,미니탕용기_미니,49800.0,60336.933827,1.211585
20,탕용기_195_小,43800.0,44413.524182,1.014007
4,중화면용기_小,36600.0,63750.929324,1.741829
52,원형용기_150ml,36000.0,36471.246173,1.013090
13,탕용기_195_大,36000.0,35611.523028,0.989209


In [62]:
real_order_amount_2019_05['비율'].median()

1.2115850166075588